#### Listing 13.1 - Computing the maximum likelihood estimate of mean and variance for a Gaussian distribution using SymPy

In [ ]:
#Listing 13.1 - Computing the maximum likelihood estimate of mean and variance for a Gaussian distribution using SymPy
import sympy as sp

N = 4
x = sp.symbols(f'x0:{N}')
print('Data observations:', x)

mu, sigma2 = sp.symbols('mu sigma2', real=True, positive=True)

L = sum(-sp.log(sp.sqrt(2 * sp.pi * sigma2)) - ((xi - mu)**2 / (2 * sigma2)) for xi in x)

dL_dmu = sp.diff(L, mu)
dL_dsigma2 = sp.diff(L, sigma2)

mu_hat = sp.solve(dL_dmu, mu)
print('The maximum likelihood estimate of mu is:\n', mu_hat)

sigma2_hat = sp.solve(dL_dsigma2, sigma2)
print('The maximum likelihood estimate of sigma2 is:\n',sigma2_hat)

#### Listing 13.2 - Example of MLE

In [ ]:
#Listing 13.2 - Example of MLE
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

data = np.array([0, 1])
mu_mle, sigma_mle = np.mean(data), np.std(data, ddof=0)
print('The maximum likelihood estimates, mu and sigma are: {} and {}.\n'. format(mu_mle, sigma_mle))

GaussDistrs = {'MLE': (mu_mle, sigma_mle), 'Gauss 1': (0, 1), 'Gauss 2': (1, 2)}
print('GaussDistrs:',GaussDistrs)
likelihoods = {key: stats.norm.pdf(data, mu, sigma) for key, (mu, sigma) in GaussDistrs.items()}
print('likelihoods:',likelihoods)

for key,lik in zip(likelihoods.keys(),likelihoods.values()):
    print('For distribution:',key)
    product = lik[0]*lik[1]
    print('\tProduct of probability density values is:',round(product,4))

x = np.linspace(-3, 4, 1000)
plt.figure(figsize=(8, 6))
for label, (mu, sigma) in GaussDistrs.items():
    plt.plot(x, stats.norm.pdf(x, mu, sigma), label=f'{label} (μ={mu:.1f}, σ={sigma:.1f})')

offset = 0.01
plt.scatter(data, [offset, offset], color='black')
for i, x_val in enumerate(data):
    plt.vlines(x_val, offset, max(lik[i] for lik in likelihoods.values()), color='gray', linestyle='dashed')
    plt.hlines([lik[i] for lik in likelihoods.values()], -3, data[i], color='gray', linestyle='dashed')

plt.xlim(-3, 4), plt.ylim(0, None)
plt.xlabel('x', fontsize ="15")
plt.ylabel("Probability Density", fontsize ="15")
plt.legend()
plt.grid()
plt.savefig('MLE.eps', format='eps', dpi=600)

#### Listing 13.3 - Linear regression with maximum likelihood Estimation

In [ ]:
#Listing 13.3 - Linear regression with maximum likelihood Estimation
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(421)
n_data, d = 50, 1
a = np.array([1,3])
X = np.random.rand(n_data, 1)
X = np.c_[np.ones(n_data), X]
noise = np.random.randn(n_data) * 0.25
y = X @ a + noise

a_mle = np.linalg.inv(X.T @ X) @ X.T @ y
sigma2_mle = np.sum((y - X @ a_mle) ** 2) / (n_data-(d+1))
print('MLE estimate of the regression coefficient vector:', np.round(a_mle,2))
print('MLE estimate of sigma^2 = :', np.round(sigma2_mle,2))

plt.scatter(X[:, 1], y)
plt.plot(X[:, 1], X @ a_mle, color='red')
plt.xlabel('X', fontsize ="12")
plt.ylabel('y', fontsize ="12")
plt.savefig('LR_MLE.eps', format='eps', dpi=600)

#### Listing 13.4 - Sampling distribution of the linear regression estimators: a - manually

In [ ]:
#Listing 13.4 - Sampling distribution of the linear regression estimators: set up

import numpy as np
import scipy.stats as stats
import statsmodels.api as sm

X = np.array([1,2,3,4,5])
X = sm.add_constant(X)
y = np.array([2.4,2,1.6,1,0.4])

n_data = len(y)
d = 1
df = n_data - (d+1)
alpha = 0.05

#### Listing 13.5

In [ ]:
#Listing 13.5 - Sampling distribution of the linear regression estimators: 
# finding confidence interval for a manually
XTX_inv = np.linalg.inv(X.T @ X)
a_mle = XTX_inv @ X.T @ y
print('The estimated a0 ={} and a1={}'.format(np.round(a_mle[0],2), np.round(a_mle[1],2)))
sigma2_mle = np.sum((y - X @ a_mle) ** 2) / (n_data-(d+1))
print('The estimated sigma^2 =:', np.round(sigma2_mle,4))

SE_a0 = np.sqrt(sigma2_mle * XTX_inv[0, 0])
SE_a1 = np.sqrt(sigma2_mle * XTX_inv[1, 1])
t_critical = stats.t.ppf(1 - alpha/2, df)
CI_a0 = (np.round((a_mle[0] - t_critical * SE_a0), 2), np.round((a_mle[0] + t_critical * SE_a0),2))
CI_a1 = (np.round((a_mle[1] - t_critical * SE_a1), 2), np.round((a_mle[1] + t_critical * SE_a1),2))

print(f'95% Confidence Interval for a0: {CI_a0}')
print(f'95% Confidence Interval for a1: {CI_a1}')

#### Listing 13.6 - Sampling distribution of the linear regression estimators: a - statsmodels

In [ ]:
#Listing 13.6 - Sampling distribution of the linear regression estimators: 
# findingconfidence interval for a using statsmodels
model = sm.OLS(y, X).fit()
print('The estimated a0 ={} and a1={}'.format(np.round(model.params[0],2), np.round(model.params[1],2)))

sigma2 = model.scale
print('The estimated sigma^2 =:', np.round(sigma2,4))

CI_params = model.conf_int(alpha=0.05)
print('The 95% confidence interval for a0 is:', np.round(CI_params[0,:], 2))
print('The 95% confidence interval for a1 is:', np.round(CI_params[1,:], 2))

#### Listing 13.7 - Sampling distribution of the linear regression estimators: sigma2 - statsmodels

In [ ]:
#Listing 13.7 - Sampling distribution of the linear regression estimators:
# finding sigma2 using statsmodels
chi2_lower = stats.chi2.ppf(1 - alpha/2, df)
chi2_upper = stats.chi2.ppf(alpha/2, df)

sigma2_lower = (df * sigma2) / chi2_lower
sigma2_upper = (df * sigma2) / chi2_upper

print('The 95% confidence interval for sigma^2 is:[{},{}]'.format(np.round(sigma2_lower,3), np.round(sigma2_upper,3)))

#### Listing 13.8 - Sampling distribution of the linear regression: prediction - statsmodels

In [ ]:
#Listing 13.8 - Sampling distribution of the linear regression: 
# finding a new prediction using statsmodels
X_new = np.array([[1,3.5]])
pred = model.get_prediction(X_new)
pred_summary = pred.summary_frame(alpha=0.05)

print(pred_summary)

In [ ]:
#Additional part for Listing 13.8
y_pred = pred_summary['mean'].iloc[0]
ci_lower, ci_upper = pred_summary["mean_ci_lower"].iloc[0], pred_summary["mean_ci_upper"].iloc[0]

print('Prediction for X_new = 3.5 is {}'.format(np.round(y_pred,2)))
print('The 95% confidence interval for the mean prediction is [{},{}]'.format(np.round(ci_lower,2), np.round(ci_upper,2)))

#### Logistic regression from scratch

In [ ]:
#Function needed for Listing 13.9
def sigmoid(z):
    return 1/(np.exp(-z)+1)

#### Listing 13.9

In [ ]:
#Listing 13.9
def predict(X, weights):
    linear_z = np.dot(X, weights)
    print('z value =', linear_z)
    probabilities = sigmoid(linear_z)

    return  probabilities

#### Listing 13.10

In [ ]:
#Listing 13.10
def grad_descent_lr(X, y, weights, iters, epsilon):
    for _ in range(iters):
        y_predicted = predict(X, weights)
        print('The probabilities before iteration:', np.round(y_predicted,3))
        initial_pred_labels = (y_predicted >= 0.5).astype(int)
        print('Original predictions are:',initial_pred_labels)
        error = y_predicted - y
        dw = np.dot(X.T, error)
        weights = weights - epsilon * dw

    return weights

#### Listing 13.11

In [ ]:
#Listing 13.11
import numpy as np

X = np.array([-1.5, -1, 0, 0.3])
X = np.c_[np.ones(X.shape[0]), X]
y = np.array([0, 0, 1, 1])

W = np.array([0.5, 0.4])
learning_rate = 0.1
num_iterations = 1

updated_W = grad_descent_lr(X, y, W, num_iterations, learning_rate)
probs = predict(X, updated_W)
pred_labels = (probs >= 0.5).astype(int)

print('The updated weights are:', np.round(updated_W,3))
print('The new probabilities are:', np.round(probs,3))
print('New predictions are:', pred_labels)
print('True Labels are:', y)


#### Listing 13.12 - Implement logistic regression using existing libraries

In [ ]:
#Listing 13.12 - Implement logistic regression using existing libraries
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(421)
n_samples, n_features = 50, 2

mean_0, cov_0 = [1, 2], [[1, 0.5], [0.5, 1]]
mean_1, cov_1 = [3, 3], [[1, -0.3], [-0.3, 1]]

data1 = np.random.multivariate_normal(mean_0, cov_0, n_samples)
data2 = np.random.multivariate_normal(mean_1, cov_1, n_samples)
X = np.vstack((data1, data2))
y = np.hstack((np.zeros(n_samples), np.ones(n_samples)))

plt.figure(figsize=(8, 6))
plt.scatter(data1[:, 0], data1[:, 1], label='Class 0')
plt.scatter(data2[:, 0], data2[:, 1], label='Class 1')
plt.xlabel('x1', fontsize ="15")
plt.ylabel('x2', fontsize ="15")
plt.legend()
plt.grid(True)
plt.savefig('data.eps', format='eps', dpi=600)

#### Listing 13.13

In [ ]:
#Listing 13.13
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('Before normalisation:')
print(f'X_train Mean: {np.mean(X_train, axis=0).round(2)}, X_train Std: {np.std(X_train, axis=0).round(2)}')
print(f'X_test Mean: {np.mean(X_test, axis=0).round(2)}, X_test Std: {np.std(X_test, axis=0).round(2)}')
print('After normalisation:')
print(f'X_train_scaled Mean: {np.mean(X_train_scaled, axis=0).round(2)}, X_train Std: {np.std(X_train_scaled, axis=0).round(2)}')
print(f'X_test_scaled Mean: {np.mean(X_test_scaled, axis=0).round(2)}, X_test Std: {np.std(X_test_scaled, axis=0).round(2)}')

#### Listing 13.14

In [ ]:
#Listing 13.14
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

sklearn_model = LogisticRegression(penalty=None, solver='lbfgs', tol=1e-8)
sklearn_model.fit(X_train_scaled, y_train)
y_pred_sklearn = sklearn_model.predict(X_test_scaled)
print('Model intercept is {} and \n coefficients are{}'.format(sklearn_model.intercept_,sklearn_model.coef_))
print('Sklearn logistic regression accuracy is:', accuracy_score(y_test, y_pred_sklearn))

probabilities = sklearn_model.predict_proba(X_test_scaled)[:, 1]
threshold = 0.5
y_pred_threshold = (probabilities >= threshold).astype(int)
print('Sklearn logistic regression accuracy at threshold 0.5 is:', accuracy_score(y_test, y_pred_threshold))

#### Listing 13.15

In [ ]:
#Listing 13.15
import statsmodels.api as sm

X_train_sm = sm.add_constant(X_train_scaled)
X_test_sm = sm.add_constant(X_test_scaled)

sm_model = sm.Logit(y_train, X_train_sm).fit(method='lbfgs')
y_pred_probs_sm = sm_model.predict(X_test_sm)
y_pred_sm = (y_pred_probs_sm >= 0.5).astype(int)
print('Model intercept & coefficients are:\n', sm_model.params)
print('Statsmodels logistic regression accuracy is:', accuracy_score(y_test, y_pred_sm))


In [ ]:
#Listing 13.15 - extra
conf_intervals = sm_model.conf_int(alpha=0.05)
print(conf_intervals)